In [1]:
import pandas as pd

In [1]:
data_path = "leading_reviewer_data/"

In [2]:
from glob import glob
reviewer_path_list = glob(data_path+"*/")

In [79]:
import re
import os
import numpy as np

In [146]:
# only use 10 reviews for each reviewer (pick out the 10 largest size of reviews)
reviewers = []
reviewers_review = []
reviewers_literature = []
for reviewer_path in reviewer_path_list:
    reviewers.append(re.search(r'leading_reviewer_data\\([a-zA-Z_-]*)\\',reviewer_path).group(1))
    current_reviewer_review = [f for f in os.listdir(reviewer_path) if '.txt' in f]
    # select the 10 largest size of reviews
    current_selected_review_index = np.argsort([os.stat(reviewer_path+f).st_size for f in os.listdir(reviewer_path) if '.txt' in f])[::-1][:10]
    current_reviewer_review_selected = list(np.array(current_reviewer_review)[ind])
    # get the review texts
    review_texts = []
    for nm in current_reviewer_review_selected:
        with open(reviewer_path+nm, encoding="utf8") as f:
            text_to_append = f.read().replace('\n',' ')
            review_texts.append(text_to_append)
    reviewers_review.append(review_texts)
    
    current_reviewer_literature = [f for f in os.listdir(reviewer_path+"training_data\\") if '.txt' in f]
    # get the literature texts
    literature_texts = []
    for nm in current_reviewer_literature:
        with open(reviewer_path+"training_data\\"+nm, encoding="utf8") as f:
            text_to_append = f.read().replace('\n',' ')
            literature_texts.append(text_to_append)
    reviewers_literature.append(literature_texts)

# Build training set and test set
## Randomly picking the test data from the whole dataset
total: 25 reviewers

each reviewer has 10 reviews and 10 literatures.

Test set: all the reviews

Training set: all the literatures

In [100]:
np.random.seed(100)

# Features

### Lexical and punctuation features

Lexical features:

The average number of words per sentence

Sentence length variation

Lexical diversity, which is a measure of the richness of the author’s vocabulary

Punctuation features:

Average number of commas, semicolons and colons per sentence

In [153]:
# create feature vectors
def LexicalFeatures(reviews_texts):
    """
    Compute feature vectors for word and punctuation features
    """
    num_reviews = len(reviews_texts)
    fvs_lexical = np.zeros((len(reviews_texts), 3), np.float64)
    fvs_punct = np.zeros((len(reviews_texts), 3), np.float64)
    for e, ch_text in enumerate(reviews_texts):
        # note: the nltk.word_tokenize includes punctuation
        tokens = nltk.word_tokenize(ch_text.lower())
        words = word_tokenizer.tokenize(ch_text.lower())
        sentences = sentence_tokenizer.tokenize(ch_text)
        vocab = set(words)
        words_per_sentence = np.array([len(word_tokenizer.tokenize(s))
                                   for s in sentences])
 
        # average number of words per sentence
        fvs_lexical[e, 0] = words_per_sentence.mean()
        # sentence length variation
        fvs_lexical[e, 1] = words_per_sentence.std()
        # Lexical diversity
        fvs_lexical[e, 2] = len(vocab) / float(len(words))
 
        # Commas per sentence
        fvs_punct[e, 0] = tokens.count(',') / float(len(sentences))
        # Semicolons per sentence
        fvs_punct[e, 1] = tokens.count(';') / float(len(sentences))
        # Colons per sentence
        fvs_punct[e, 2] = tokens.count(':') / float(len(sentences))

    # apply whitening to decorrelate the features
    fvs_lexical = whiten(fvs_lexical)
    fvs_punct = whiten(fvs_punct)
    
    return fvs_lexical, fvs_punct

## bag of words feature

In [154]:
def BagOfWords(all_reviews_texts, reviews_texts):
    """
    Compute the bag of words feature vectors, based on the most common words
     in the whole book
    """
    # get most common words in the whole book
    NUM_TOP_WORDS = 10
    all_tokens = nltk.word_tokenize(all_reviews_texts)
    fdist = nltk.FreqDist(all_tokens)
    vocab = list(fdist.keys())[:NUM_TOP_WORDS]
 
    # use sklearn to create the bag for words feature vector for each chapter
    vectorizer = CountVectorizer(vocabulary=vocab, tokenizer=nltk.word_tokenize)
    fvs_bow = vectorizer.fit_transform(reviews_texts).toarray().astype(np.float64)
 
    # normalise by dividing each row by its Euclidean norm
    fvs_bow /= np.c_[np.apply_along_axis(np.linalg.norm, 1, fvs_bow)]
    
    # deal with nan
    #Obtain mean of columns as you need, nanmean is just convenient.
    col_mean = np.nanmean(fvs_bow, axis=0)
    #Find indicies that you need to replace
    inds = np.where(np.isnan(fvs_bow))
    #Place column means in the indices. Align the arrays using take
    fvs_bow[inds] = np.take(col_mean, inds[1])
    
    return fvs_bow

## Syntactic features

In [155]:
def SyntacticFeatures(reviews_texts):
    """
    Extract feature vector for part of speech frequencies
    """
    # get part of speech for each token in each review
    def token_to_pos(ch):
        tokens = nltk.word_tokenize(ch)
        return [p[1] for p in nltk.pos_tag(tokens)]
    review_pos = [token_to_pos(ch) for ch in reviews_texts]
 
    # count frequencies for common POS types
    pos_list = ['NN', 'NNP', 'DT', 'IN', 'JJ', 'NNS']
    fvs_syntax = np.array([[ch.count(pos) for pos in pos_list] for ch in review_pos]).astype(np.float64)
 
    # normalise by dividing each row by number of tokens in the review
    fvs_syntax /= np.c_[np.array([len(ch) for ch in review_pos])]
    
    return fvs_syntax

# Classifier

## Baseline: logistic regression

In [212]:
import nltk
from scipy.cluster.vq import whiten
from sklearn.feature_extraction.text import CountVectorizer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

training data

In [215]:
fvs_lexical_train = np.array([], dtype=np.int64).reshape(0,3)
fvs_punct_train = np.array([], dtype=np.int64).reshape(0,3)
fvs_bow_train = np.array([], dtype=np.int64).reshape(0,10)
fvs_syntax_train = np.array([], dtype=np.int64).reshape(0,6)
true_label_train = np.array([], dtype=np.int64).reshape(0,1)
for lit in np.arange(len(reviewers_literature)):
    # lexical and punct
    fvs_lexical_train_cur, fvs_punct_train_cur = LexicalFeatures(reviewers_literature[lit])
    fvs_lexical_train = np.concatenate((fvs_lexical_train,fvs_lexical_train_cur))
    fvs_punct_train = np.concatenate((fvs_punct_train,fvs_punct_train_cur))
    # bag of words
    all_literature_texts = ' '.join(reviewers_literature[lit])
    fvs_bow_train_cur = BagOfWords(all_literature_texts, reviewers_literature[lit])
    fvs_bow_train = np.concatenate((fvs_bow_train,fvs_bow_train_cur))
    # syntax
    fvs_syntax_train_cur = SyntacticFeatures(reviewers_literature[lit])
    fvs_syntax_train = np.concatenate((fvs_syntax_train,fvs_syntax_train_cur))
    # label
    true_label_train = np.concatenate((true_label_train, lit*np.ones(( len(reviewers_literature[lit]) ,1))))

In [223]:
fvs_train = np.concatenate((fvs_lexical_train,fvs_punct_train,fvs_bow_train,fvs_syntax_train),axis=1)